In [1]:
import pandas as pd
import numpy as np
import gc
from tqdm.notebook import tqdm_notebook
import open3d as o3d
import copy
import matplotlib.pyplot as plt
import transforms3d
import os

In [2]:
%matplotlib notebook

In [3]:
def draw_registration_result(src_pcd, dst_pcd, x_pt, y_pt, theta):    
    src_pcd_tmp = copy.deepcopy(src_pcd)
    dst_pcd_tmp = copy.deepcopy(dst_pcd)
    
    src_pcd_tmp.paint_uniform_color([1, 0, 0])  # red source
    dst_pcd_tmp.paint_uniform_color([0, 0, 1])  # blue target
    
    transform_mat = pose2matrix([x_pt, y_pt, 0], [0,0,theta])
    dst_pcd_tmp.transform(transform_mat)
    
    visualizer = o3d.JVisualizer()
    visualizer.add_geometry(src_pcd_tmp)
    visualizer.add_geometry(dst_pcd_tmp)
    visualizer.show()
    
def draw_pcd(pcd, where='opn_nb'):
    if where is 'opn_nb':
        visualizer = o3d.JVisualizer()
        visualizer.add_geometry(pcd)
        visualizer.show()
    elif where is 'opn_view':
        o3d.visualization.draw_geometries([pcd], width=1280, height=800)
    elif where is 'mat_3d':
        plt.figure()
        pts = np.asarray(pcd.points)
        plt.scatter(pts[:,0], pts[:,1], pts[:,2])
        plt.grid()
        plt.show()
    elif where is 'mat_2d':
        plt.figure()
        pts = np.asarray(pcd.points)
        plt.scatter(pts[:,0], pts[:,1])
        plt.grid()
        plt.show()

def pose2matrix(translation_list, rotation_angle_list):
    trans_vec = np.array(translation_list)
    rot_ang = [np.deg2rad(ang) for ang in rotation_angle_list ]
    rot_mat = transforms3d.euler.euler2mat(rot_ang[0], rot_ang[1], rot_ang[2])
    zoom = np.ones(3)
    transform_mat = transforms3d.affines.compose(trans_vec, rot_mat, zoom)
    return transform_mat

### GT filtering : Extract the GT mask from segment pcd 

In [4]:
BASE_TESTING_PATH = "/home/saby/Projects/ati/data/data/datasets/Carla/16beam-Data/small_map/testing"
FOLDER_OF_INTEREST = "8"

DYNAMIC_PCD_FOLDER = "_out"
DYNAMIC_NPY_FOLDER = "_out_out_npy"
SEGMENT_PCD_FOLDER = "_segment"
SEGMENT_NPY_FOLDER = "_segment_out_npy"
CORR_STATIC_PCD_FOLDER = "corr_static"
CORR_STATIC_NPY_FOLDER = "corr_static_out_npy"
# UNET_PCD_FOLDER = "unet_pcd"
# UNET_NPY_FOLDER = "unet_pcd_out_npy"
UNET_LEARN_FILTER_PCD_FOLDER = "first_new_attempt_new_unet_64f_gen_125_pcd"
UNET_LEARN_FILTER_NPY_FOLDER = "first_new_attempt_new_unet_64f_gen_125_pcd_out_npy"

TEST_PATH = os.path.join(BASE_TESTING_PATH, FOLDER_OF_INTEREST)
TEST_DYNAMIC_PCD_PATH = os.path.join(TEST_PATH, DYNAMIC_PCD_FOLDER)
TEST_DYNAMIC_NPY_PATH = os.path.join(TEST_PATH, DYNAMIC_NPY_FOLDER)
TEST_SEGMENT_PCD_PATH = os.path.join(TEST_PATH, SEGMENT_PCD_FOLDER)
TEST_SEGMENT_NPY_PATH = os.path.join(TEST_PATH, SEGMENT_NPY_FOLDER)
TEST_CORR_STATIC_PCD_PATH = os.path.join(TEST_PATH, CORR_STATIC_PCD_FOLDER)
TEST_CORR_STATIC_NPY_PATH = os.path.join(TEST_PATH, CORR_STATIC_NPY_FOLDER)
# TEST_UNET_PCD_PATH = os.path.join(TEST_PATH, UNET_PCD_FOLDER)
# TEST_UNET_NPY_PATH = os.path.join(TEST_PATH, UNET_NPY_FOLDER)
TEST_UNET_LEARN_FILTER_PCD_PATH = os.path.join(TEST_PATH, UNET_LEARN_FILTER_PCD_FOLDER)
TEST_UNET_LEARN_FILTER_NPY_PATH = os.path.join(TEST_PATH, UNET_LEARN_FILTER_NPY_FOLDER)


In [5]:
segment_arr = np.load(os.path.join(TEST_SEGMENT_NPY_PATH, "0.npy"))
segment_arr.shape

(2827, 16, 1024, 4)

In [6]:
corr_static_arr = np.load(os.path.join(TEST_CORR_STATIC_NPY_PATH, "0.npy"))
corr_static_arr.shape

(2827, 16, 1024, 4)

In [7]:
dynamic_arr = np.load(os.path.join(TEST_DYNAMIC_NPY_PATH, "0.npy"))
dynamic_arr.shape

(2827, 16, 1024, 4)

### Rule Based Manual Filtering (Baseline)

#### Optimal filter value range for each pcd in the run

#### Overall best optimal filter value on complete run for manual filtering 

### Model filtering

#### Extract the mask by learnt filter models

In [8]:
unet_learn_filter_arr = np.load(os.path.join(TEST_UNET_LEARN_FILTER_NPY_PATH, "0.npy"))
unet_learn_filter_arr.shape

(2827, 16, 1024, 4)

In [9]:
idx = np.random.choice(unet_learn_filter_arr.shape[0], 1)[0]
idx

1877

In [10]:
segment_pcd = o3d.io.read_point_cloud(os.path.join(TEST_SEGMENT_PCD_PATH, str(idx+1)+".ply"))
segment_pcd

geometry::PointCloud with 20855 points.

In [11]:
corr_static_pcd = o3d.io.read_point_cloud(os.path.join(TEST_CORR_STATIC_PCD_PATH, str(idx+1)+".ply"))
corr_static_pcd

geometry::PointCloud with 20722 points.

In [12]:
print("Dynamic PCD with segmentation")
draw_pcd(segment_pcd)

Dynamic PCD with segmentation


JVisualizer with 1 geometries

In [13]:
print("Static PCD corresponding")
draw_pcd(corr_static_pcd)

Static PCD corresponding


JVisualizer with 1 geometries

In [33]:
learn_mask_img = unet_learn_filter_arr[idx,:,:,3]
learn_mask_flat = learn_mask_img.reshape((-1,1))
learn_mask_img.shape, learn_mask_flat.shape

((16, 1024), (16384, 1))

In [34]:
set(learn_mask_flat.flatten())

{0.0, 1.0}

In [16]:
# learn_mask_flat = np.array([int(not item) for item in learn_mask_flat]).reshape(-1,1) # inverting learn_mask_flat
learn_mask_flat = learn_mask_flat
color_arr = np.concatenate((learn_mask_flat, np.zeros(learn_mask_flat.shape), np.zeros(learn_mask_flat.shape)), axis=1)
color_arr.shape

(16384, 3)

In [17]:
recon_img = unet_learn_filter_arr[idx,:,:,:3] * 100
recon_flat = recon_img.reshape((-1,3))
recon_img.shape, recon_flat.shape

((16, 1024, 3), (16384, 3))

In [18]:
recon_pcd = o3d.geometry.PointCloud()
recon_pcd.points = o3d.utility.Vector3dVector(recon_flat)
recon_pcd.colors = o3d.utility.Vector3dVector(color_arr)
recon_pcd

geometry::PointCloud with 16384 points.

In [19]:
dynamic_img = dynamic_arr[idx,:,:,:3]
dynamic_img.shape

(16, 1024, 3)

In [20]:
learn_mask_expand = learn_mask_img.reshape((learn_mask_img.shape[0],learn_mask_img.shape[1],1))
learn_mask_expand.shape

(16, 1024, 1)

In [21]:
new_recon_img = (dynamic_img * (learn_mask_expand)) + ((1-learn_mask_expand) * recon_img)
new_recon_flat = new_recon_img.reshape((-1,3))
new_recon_img.shape, new_recon_flat.shape

((16, 1024, 3), (16384, 3))

In [22]:
dynamic_w_learn_mask_pcd = o3d.geometry.PointCloud()
dynamic_w_learn_mask_pcd.points = o3d.utility.Vector3dVector(dynamic_img.reshape((-1,3)))
dynamic_w_learn_mask_pcd.colors = o3d.utility.Vector3dVector(color_arr)
dynamic_w_learn_mask_pcd

geometry::PointCloud with 16384 points.

In [23]:
print("Dynamic with learnt filter PCD")
draw_pcd(dynamic_w_learn_mask_pcd)

Dynamic with learnt filter PCD


JVisualizer with 1 geometries

In [24]:
dynamic_w_learn_mask_pcd

geometry::PointCloud with 16384 points.

In [25]:
new_recon_pcd = o3d.geometry.PointCloud()
new_recon_pcd.points = o3d.utility.Vector3dVector(new_recon_flat)
new_recon_pcd.colors = o3d.utility.Vector3dVector(color_arr)
new_recon_pcd

geometry::PointCloud with 16384 points.

In [26]:
print("Pure reconstruction PCD")
draw_pcd(recon_pcd)

Pure reconstruction PCD


JVisualizer with 1 geometries

In [27]:
print("Filtered reconstruction PCD")
draw_pcd(new_recon_pcd)

Filtered reconstruction PCD


JVisualizer with 1 geometries

In [28]:
andandand

NameError: name 'andandand' is not defined

In [ ]:
(dynamic_arr - corr_static_arr).shape

In [ ]:
# wt_mask = 
dynamic_arr[:,:,:,2]

In [ ]:
ATH = os.path.join(TEST_PATH, UNET_PCD_FOLDER)
# TEST_UNET_NPY_PATH = os.path.join(TEST_PATH, UNET_PCD_FOLDER)
# TEST_UNET_NPY_P

#### Rule based Filtering applied on Model filtered outputs

### Compare the Baseline Manual FIltering and Model Filtering against GT filtering

### Compare on SLAM performance